In [16]:
import pyarrow.parquet as pq
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

from collections import defaultdict

import warnings

warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

# 데이터 불러오기
# Parquet 파일 열기
table = pq.read_table('../data/train.parquet')

# pandas DataFrame으로 변환
data_raw = table.to_pandas() 

def is_purchase(x):
    if x =='purchase':
        return 1
    else:
        return 0

data_raw.loc[:,'purchase_count'] = data_raw['event_type'].apply(is_purchase)

In [17]:
user_raw = data_raw.groupby('user_id').sum('purchase_count').reset_index()
view_user = user_raw[user_raw['purchase_count']==0] # view, cart가 있는 user
cp_user = user_raw[user_raw['purchase_count']!=0]   # pusrchase만 있는 user

In [58]:
view_user.shape, cp_user.shape

((636576, 4), (1681, 4))

In [2]:
import pandas as pd
train_total = pd.read_csv('../data/total_train.csv')

In [60]:
only_view_data = view_user['user_id'].to_frame().merge(data_raw, on='user_id', how='inner') 
top_items_per_user = (
    only_view_data.groupby('user_id')['item_id']
    .value_counts()  # user_id 별 item_id의 빈도 계산
    .groupby(level=0)  # user_id를 기준으로 그룹화
    .head(3)  # 상위 3개 아이템만 선택
    .reset_index(name='count')  # 데이터프레임으로 변환
)

top_items_per_user

,user_id,item_id,count
0,0000162d-72b9-4b0f-8895-7b52b6661c82,0bb05529-df13-46ca-8bfc-832344a7b1d3,3
1,0000162d-72b9-4b0f-8895-7b52b6661c82,a9e8c988-8b93-4ea2-8a97-b9ef898342dd,3
2,0000162d-72b9-4b0f-8895-7b52b6661c82,383fde85-a76b-447d-ab42-0e26e4897420,2
3,00001710-fd5d-47e4-aecb-8da4173a7c02,838f5917-138a-4e84-8e98-e75b5622c9b6,1
4,00002da8-29b8-4d15-bf99-b4b4475e4631,2e0d2baf-84f8-4609-8411-40106770595a,2
...,...,...,...
1596688,ffffc371-aee8-4d93-9327-fdb777037789,2f53a941-e626-41b3-bd64-3f0cf6cb17ee,1
1596689,ffffc371-aee8-4d93-9327-fdb777037789,5e6a67eb-3cb2-4184-9db5-d2c5a9afd649,1
1596690,ffffc959-2575-476c-b897-057e51182c5f,bdc5f1cb-ce06-47d6-9d3f-f98d79bf54e1,1
1596691,ffffc959-2575-476c-b897-057e51182c5f,b6b56d28-0f9c-4906-93df-cab0532930b8,1


In [61]:
temp1 = top_items_per_user.merge(train_total,on=['user_id','item_id'],how='inner').drop('count',axis=1)
temp1

,user_id,item_id,event_weight,brand,price,date_weight,frequency,monetary,cluster,brand_encoded
0,0000162d-72b9-4b0f-8895-7b52b6661c82,0bb05529-df13-46ca-8bfc-832344a7b1d3,0.000663,apple,643.23,0,28,0.0,0.0,84
1,0000162d-72b9-4b0f-8895-7b52b6661c82,a9e8c988-8b93-4ea2-8a97-b9ef898342dd,0.000663,apple,553.14,0,28,0.0,0.0,84
2,0000162d-72b9-4b0f-8895-7b52b6661c82,383fde85-a76b-447d-ab42-0e26e4897420,0.000442,apple,640.92,0,28,0.0,0.0,84
3,00001710-fd5d-47e4-aecb-8da4173a7c02,838f5917-138a-4e84-8e98-e75b5622c9b6,0.000221,machete,38.61,5,1,0.0,0.0,1014
4,00002da8-29b8-4d15-bf99-b4b4475e4631,2e0d2baf-84f8-4609-8411-40106770595a,0.000442,xiaomi,33.15,0,11,0.0,0.0,1823
...,...,...,...,...,...,...,...,...,...,...
1596688,ffffc371-aee8-4d93-9327-fdb777037789,2f53a941-e626-41b3-bd64-3f0cf6cb17ee,0.000221,defacto,12.84,5,9,0.0,0.0,431
1596689,ffffc371-aee8-4d93-9327-fdb777037789,5e6a67eb-3cb2-4184-9db5-d2c5a9afd649,0.000221,sony,447.63,5,9,0.0,0.0,1586
1596690,ffffc959-2575-476c-b897-057e51182c5f,bdc5f1cb-ce06-47d6-9d3f-f98d79bf54e1,0.000221,pt-group,231.67,5,6,0.0,0.0,1369
1596691,ffffc959-2575-476c-b897-057e51182c5f,b6b56d28-0f9c-4906-93df-cab0532930b8,0.000221,pt-group,231.67,5,6,0.0,0.0,1369


In [62]:
temp2 = cp_user['user_id'].to_frame().merge(train_total,on='user_id',how='inner')
temp2

,user_id,item_id,event_weight,brand,price,date_weight,frequency,monetary,cluster,brand_encoded
0,002427d7-3da1-4d79-aca9-32f063b179bc,2e0d2baf-84f8-4609-8411-40106770595a,0.000442,xiaomi,33.33,5,21,51.46,0.0,1823
1,002427d7-3da1-4d79-aca9-32f063b179bc,43f03e17-572a-41e9-b6c6-4c3948d48542,0.887431,honor,51.46,5,21,51.46,0.0,767
2,002427d7-3da1-4d79-aca9-32f063b179bc,5412e9c1-6014-4dc0-afe1-07d824a4868c,0.000663,samsung,25.46,5,21,51.46,0.0,1483
3,002427d7-3da1-4d79-aca9-32f063b179bc,5b071914-719c-4a73-bbd1-ab8100ba1eb2,0.000884,samsung,25.46,5,21,51.46,0.0,1483
4,002427d7-3da1-4d79-aca9-32f063b179bc,5efd5ef4-7b4a-48e3-85dd-61871ab53c4b,0.000663,samsung,59.40,5,21,51.46,0.0,1483
...,...,...,...,...,...,...,...,...,...,...
36705,ffd9366a-3bf3-4f34-89a9-01dc2c76a6f8,cd127454-7837-4095-b055-77619492fbf3,0.000221,amd,2257.63,5,44,25.71,0.0,61
36706,ffd9366a-3bf3-4f34-89a9-01dc2c76a6f8,d32daeef-9138-4964-9626-64fe5004b55e,0.000221,amd,147.23,5,44,25.71,0.0,61
36707,ffd9366a-3bf3-4f34-89a9-01dc2c76a6f8,d98b9f41-73b9-481b-a5a1-d0d469d24ecd,0.000221,amd,354.96,5,44,25.71,0.0,61
36708,ffd9366a-3bf3-4f34-89a9-01dc2c76a6f8,e165f519-46e3-452d-b991-9a6026e45326,0.000221,gigabyte,118.56,5,44,25.71,0.0,671


### merge & save

In [64]:
pd.concat([temp2,temp1]).to_csv('../data/reduce_total_train.csv',index=False)

,user_id,item_id,event_weight,brand,price,date_weight,frequency,monetary,cluster,brand_encoded
0,002427d7-3da1-4d79-aca9-32f063b179bc,2e0d2baf-84f8-4609-8411-40106770595a,0.000442,xiaomi,33.33,5,21,51.46,0.0,1823
1,002427d7-3da1-4d79-aca9-32f063b179bc,43f03e17-572a-41e9-b6c6-4c3948d48542,0.887431,honor,51.46,5,21,51.46,0.0,767
2,002427d7-3da1-4d79-aca9-32f063b179bc,5412e9c1-6014-4dc0-afe1-07d824a4868c,0.000663,samsung,25.46,5,21,51.46,0.0,1483
3,002427d7-3da1-4d79-aca9-32f063b179bc,5b071914-719c-4a73-bbd1-ab8100ba1eb2,0.000884,samsung,25.46,5,21,51.46,0.0,1483
4,002427d7-3da1-4d79-aca9-32f063b179bc,5efd5ef4-7b4a-48e3-85dd-61871ab53c4b,0.000663,samsung,59.40,5,21,51.46,0.0,1483
...,...,...,...,...,...,...,...,...,...,...
1596688,ffffc371-aee8-4d93-9327-fdb777037789,2f53a941-e626-41b3-bd64-3f0cf6cb17ee,0.000221,defacto,12.84,5,9,0.00,0.0,431
1596689,ffffc371-aee8-4d93-9327-fdb777037789,5e6a67eb-3cb2-4184-9db5-d2c5a9afd649,0.000221,sony,447.63,5,9,0.00,0.0,1586
1596690,ffffc959-2575-476c-b897-057e51182c5f,bdc5f1cb-ce06-47d6-9d3f-f98d79bf54e1,0.000221,pt-group,231.67,5,6,0.00,0.0,1369
1596691,ffffc959-2575-476c-b897-057e51182c5f,b6b56d28-0f9c-4906-93df-cab0532930b8,0.000221,pt-group,231.67,5,6,0.00,0.0,1369
